In [4]:
%matplotlib wx
import superball_multilateration_barconstraints as smb
import numpy as np 
import rospy
import rosbag
from matplotlib import pylab as plt
plt.ion()
from sklearn import linear_model, datasets

In [27]:
topics = ['/bbb10/0x2017_0x1','/bbb10/0x2017_0x2','/bbb10/0x2017_0x4','/bbb10/0x2017_0x7','/bbb10/0x2017_0x8',
          '/bbb10/0x2017_0xc','/bbb10/0x2017_0xd','/bbb10/0x2017_0xe','/bbb10/0x2017_0x10',
         ]
topics_node_to_node = np.array([[10,1],
                               [10,2],
                               [10,4],
                               [10,7],
                               [10,8],
                                [10,12],
                                [10,13],
                                [10,14],
                                [10,16]
                               ],dtype=int)

#topics_sensor = np.zeros(topics_node_to_node.shape,dtype=int)
#for i in xrange(topics_node_to_node.shape[0]):
#    topics_sensor[i] = (node_to_sensor[topics_node_to_node[i,0]],node_to_sensor[topics_node_to_node[i,1]])

#1,2,4,7,8,10
node_indices = {1:0,2:1,4:2,7:3,8:4,10:5,12:6,13:7,14:8,16:9}
bag_names = []
measured_distances = []
node_to_sensor = []

bag_names.append("FUN_PENDULUM_2015-08-14-16-25-39.bag")
node_to_sensor.append({1:1,2:5,4:3,7:6,8:4,10:2,12:9,13:13,14:14,16:16})

bag_names.append("SWING_PENDULUM_2015-08-14-16-16-08.bag")
node_to_sensor.append({1:1,2:5,4:3,7:6,8:4,10:2,12:9,13:13,14:14,16:16})

bag_names.append('2D_SWING_PENDULUM_2015-08-14-16-18-23.bag')
node_to_sensor.append({1:1,2:5,4:3,7:6,8:4,10:2,12:9,13:13,14:14,16:16})

In [53]:
#process all bag files
plt.close('all')
bag_means = []
bag_std = []
bag_means_raw = []
bag_stds_raw = []
bag_means_nodes = []
bag_std_nodes = []
data_clean = []
for n_i,n in enumerate(bag_names):
    print n
    bag = rosbag.Bag(n)
    data = {}
    topics_mean = np.zeros(len(topics))
    topics_std = np.zeros(len(topics))
    plt.figure()
    d_supahclean = []
    for i,t in enumerate(topics):
        plt.subplot(3,3,i+1)
        data = []
        for m in bag.read_messages(topics=[t]):
            data.append((m[1].header.stamp.to_sec(),m[1].data))
        d = np.array(data)
        d_clean = d[np.where((d[:,1]>3.5)&(d[:,1]<10))[0]] #remove bad measurements
        if(d_clean.shape[0]>10):
            model_ransac = linear_model.RANSACRegressor(linear_model.LinearRegression())
            model_ransac.fit(np.arange(d_clean.shape[0]).reshape((-1,1)), d_clean)
            inlier_mask = model_ransac.inlier_mask_
            outlier_mask = np.logical_not(inlier_mask)
            m = d_clean[inlier_mask,1].mean()
            std = d_clean[inlier_mask,1].std()
            d_outlier = d_clean[np.where((np.abs(d_clean[:,1]-m)<3*std))[0],:]
            topics_mean[i] = d_outlier[:,1].mean()
            topics_std[i] = d_outlier[:,1].std()
            #plt.plot(d_clean)
            #plt.plot(d_clean[inlier_mask])
            #plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1],'.')
            #plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1])
            
            d = d_outlier.copy()
            good_data = []
            good_data.append(d[0])
            v_d = np.zeros(d.shape)
            for i in xrange(1,d.shape[0]):
                t_1 = good_data[-1][0]
                t = d[i,0]
                p_1 = good_data[-1][1]
                p = d[i,1]
                v = np.abs((p-p_1)/(t-t_1))
                v_d[i]=v
                if(v<1.5):
                    good_data.append(d[i])
            good_data=np.array(good_data)
            plt.plot(d_outlier[:,0]-d_outlier[0,0],d_outlier[:,1],'+')
            plt.plot(good_data[:,0]-d_outlier[0,0],good_data[:,1],'.')
            plt.plot(good_data[:,0]-d_outlier[0,0],good_data[:,1],'-')
            d_supahclean.append(good_data.copy())
        else:
            topics_mean[i] = 0
            topics_std[i] = 0
            d_supahclean.append(np.zeros((0,0)))
    data_clean.append(d_supahclean)


FUN_PENDULUM_2015-08-14-16-25-39.bag
SWING_PENDULUM_2015-08-14-16-16-08.bag
2D_SWING_PENDULUM_2015-08-14-16-18-23.bag


In [93]:
#convert to clean datasets (>3 measurements), fixed rate
meas = []
for n_i,n in enumerate(bag_names):
    data = data_clean[n_i]
    t0 = np.min([d[0,0] for d in data])
    tend = np.min([d[-1,0] for d in data])
    timesteps = np.arange(t0,tend,0.2)
    measurements = np.zeros((timesteps.shape[0],len(topics)))
    
    for t_i, t in enumerate(timesteps):
        for j in xrange(len(topics)):
            d = data[j]
            #get all matching data
            idx = np.where((d[:,0]>=t) & (d[:,0]<t+0.2))[0]
            if(idx.shape>0):
                measurements[t_i,j] = np.median(d[idx,1])
            else:
                measurements[t_i,j] = 0 
    meas.append(measurements[np.where(np.sum((measurements>1) & (measurements<10),1)>=7)]) #how many valid measurements do we need?
    

In [72]:
for n_i,n in enumerate(bag_names):
    plt.figure()
    m = meas[n_i]
    print m.shape
    for i,t in enumerate(topics):
        plt.subplot(3,3,i+1)
        plt.plot(m[:,i],'.-')

(51, 9)
(41, 9)
(61, 9)


In [73]:
plt.close('all')

In [94]:
#create "variables"
def cost_function(x,y,m,o):
    '''
        Assumes x and y are Nx3, m and o are 1d vectors
    '''
    dist_xy = np.sqrt(np.sum((x-y)**2,1)) #1d vector
    cost_per_element = (dist_xy-(m-o))**2
    total_cost = cost_per_element.sum()
    return total_cost

def derivative_cost_function(x,y,m,o):
    '''
        Derivatives of the cost function wrt x,y and o
    '''
    dist_xy = np.sqrt(np.sum((x-y)**2,1)) #1d vector
    factor = 2*(dist_xy-(m-o))/dist_xy
    #derivative with respect to x
    deriv_x = factor.reshape((-1,1))*(x-y)
    #derivative with respect to y
    deriv_y = -deriv_x
    #derivative with respect to o
    deriv_o = factor*dist_xy
    return deriv_x, deriv_y, deriv_o
def create_data(fixed_nodes,floating_nodes,measurement_pairs,measurements):
    '''
        fixed nodes: indices in measurement_pairs corresponding to fixed nodes
        floating nodes: indices in measurement_paris correspond to floating nodes
        measurements pairs: each column in measurements corresponds to a distance between two nodes as specified by the rows in this matrix
        measurements: num_measurements x num pairs observed distances
        
    '''
    num_fixed = fixed_nodes.shape[0]
    num_floating = floating_nodes.shape[0]
    measurement_pairs = measurement_pairs.copy()
    
    #DOF = num_fixed #number of coordinates to find
    #num_offsets = num_fixed*num_floating#number of offset variables to find (vector rows = fixed, columns = floating -> ravel)
    
    constraints = []
    
    variables = []
    offsets = []
    
    for v in fixed_nodes:
        variables.append((v,-1))
    
    #N = [ [x y z]_fixed[0] ... [x y z]_fixed[num_fixed-1] [x y z]_float[0,meas[0]] ...[x y z]_float[0,meas[k]] [x y z]_float[1,meas[0]]...
    floating_node_to_coordinate_indices = [] #contains the indices in N for each valid measurement of a floating node
    for i in xrange(num_floating):
        
        #select all measurement pair columns to/from this node
        node = floating_nodes[i]
        
        node_dist_idx = np.where((measurement_pairs[:,0] == node) | (measurement_pairs[:,1] == node))[0]
        #print i
        #print node_dist_idx
        #select all relevant measurements (columns)
        node_meas = measurements[:,node_dist_idx]
        #only keep valid measurements 
        node_meas_check = (node_meas>1) & (node_meas<20) #conservative for now
        node_valid_distances = node_meas_check.sum(1)
        #indices in node_meas
        #select valid rows with a
        node_meas_valid = np.where((node_valid_distances>=3))[0] #only keep measurements with at least 4 valid distances
        
        for row_i,row in enumerate(node_meas_valid):
            meas_row = measurements[row]
            valid_cols = np.where(node_meas_check[row])[0]
            valid_pairs = measurement_pairs[node_dist_idx[valid_cols],:]
            meas_valid_row = meas_row[valid_cols]
            #if(node==6):
            #print node_dist_idx
            #print valid_pairs
            #print measurement_pairs[node_dist_idx[valid_cols],:]
            #print ""
            for col_i, col in enumerate(valid_cols):
                pair = np.sort(valid_pairs[col_i])
                
                #make sure we have allocated a variable for each measurement
                
                if(pair[0] in fixed_nodes):
                    if(not (pair[0],-1) in variables):
                        variables.append((pair[0],-1))
                elif(not (pair[0],row) in variables): #create new variable for this observation?
                    variables.append((pair[0],row))
                
                if(pair[1] in fixed_nodes):
                    if(not (pair[1],-1) in variables):
                        variables.append((pair[0],-1))
                elif(not tuple((pair[1],row)) in variables): #create new variable for this observation?
                    variables.append((pair[1],row)) 
                #is there an existing offset variable for this measurement pair?
                if(not tuple(pair) in offsets):
                    offsets.append(tuple(pair))
                    
                if(pair[0] in fixed_nodes):
                    v0 = (pair[0],-1)
                else:
                    v0 = (pair[0],row)
                if(pair[1] in fixed_nodes):
                    v1 = (pair[1],-1)
                else:
                    v1 = (pair[1],row)
                
                if(pair[0] in floating_nodes and pair[1] in floating_nodes):
                        print "floating node to floating node measurements"
                
                m = meas_row[node_dist_idx[col]]
                constraints.append((variables.index(v0),variables.index(v1),m,offsets.index(tuple(pair))))
                
                    

        #floating_node_to_coordinate_indices.append(node_meas_valid+DOF)    
        #DOF += node_meas_valid.shape[0] #every valid measurement adds a coordinate (3D)
    
    DOF = len(variables)
    num_offsets = len(offsets)
    #print variables
    #print offsets
    
    N_to_CSTR_X = np.zeros((len(constraints),DOF))
    N_to_CSTR_Y = np.zeros(N_to_CSTR_X.shape)
    o_to_CSTR = np.zeros((len(constraints),num_offsets))
    #print num_offsets
    m = np.zeros(len(constraints))
    for i in xrange(len(constraints)):
        N_to_CSTR_X[i,constraints[i][0]] = 1
        N_to_CSTR_Y[i,constraints[i][1]] = 1
        m[i] = constraints[i][2]
        #print constraints[i]
        o_to_CSTR[i,constraints[i][3]] = 1
    
    
    
    def deriv_func(N,o):
        X = N_to_CSTR_X.dot(N)
        Y = N_to_CSTR_Y.dot(N)
        O = o_to_CSTR.dot(o.reshape((num_offsets,1))) 
        dX,dY,dO = derivative_cost_function(X,Y,m.ravel(),O.ravel())
        dN = N_to_CSTR_X.T.dot(dX) +  N_to_CSTR_Y.T.dot(dY)
        dN[0,:] = 0
        dN[1,1:] = 0
        dN[2,2] = 0
        do = o_to_CSTR.T.dot(dO.reshape((-1,1))).ravel()
        return dN,do
    
    def cost_func(N,o):
        X = N_to_CSTR_X.dot(N)
        Y = N_to_CSTR_Y.dot(N)
        O = o_to_CSTR.dot(o.reshape((num_offsets,1))) 
        return cost_function(X,Y,m.ravel(),O.ravel())
        
    
    #create m vector
    return deriv_func,cost_func, len(variables), len(offsets)

In [271]:
#WORKING VERSION
fixed_nodes = np.array([1,2,4,7,8,12,13,16],dtype=int)
floating_nodes = np.array([10,],dtype=int)
dist_pairs = np.array([[1,10],[2,10],[4,10],[7,10],[8,10],[12,10],[13,10],[16,10]],dtype=int)

m = np.vstack((meas[0][::2],meas[1][::3]))[:,(0,1,2,3,4,5,6,8)]
#m = np.hstack((m,np.ones((m.shape[0],1))*1.36))
df,cf,DOF,num_offsets = create_data(fixed_nodes,floating_nodes,dist_pairs,m)

N,o = np.random.random((DOF,3))*5,np.zeros(num_offsets)+3.5
N[0] = 0
N[1,1:] = 0
N[2,2] = 0     
B=np.array([[ 0.,          0.,          0.        ],
                 [ 1.36230428,  0.,          0.        ],
                 [ 2.47776388,  1.21785478,  0.        ]])
N[:3] = B

dN,do = df(N,o)
print cf(N,o)

import scipy.optimize as so
Nf = N.ravel()
X_initial = np.hstack((Nf,o))
def cost_fun(x):
    Nt = x[:-o.shape[0]].reshape((DOF,3))
    Nt[:3] = B
    reg = 0*0.05*np.sum((x[-o.shape[0]:]-3.5)**2)
    return cf(Nt ,x[-o.shape[0]:]*0+3.5)+reg
def cost_grad(x):
    dN,do = df(x[:-o.shape[0]].reshape((DOF,3)),x[-o.shape[0]:])
    dN[:3] = 0
    do[:] = 0
    #do += 0.05*2*(x[-o.shape[0]:]-3.5)
    return np.hstack((dN.ravel(),do))

res = so.fmin_bfgs(cost_fun,X_initial,fprime=cost_grad,disp=10,gtol=1e-7)
print cost_fun(res)
#    #print res[:-o.shape[0]].reshape((DOF,3))[:N_target.shape[0]].round(2)
#    #print N_target.round(2)
#    #print np.sort(res[-o.shape[0]:].round(2))
#    #print np.sort(o_target.round(2))
#all_results.append((o_target.copy(),res[-o.shape[0]:].copy(),cost_fun(res)))

4823.47449355
Optimization terminated successfully.
         Current function value: 6.572584
         Iterations: 201
         Function evaluations: 230
         Gradient evaluations: 230
6.57258429849


In [272]:
res[:-o.shape[0]].reshape((DOF,3))[:fixed_nodes.shape[0]].round(2)

array([[ 0.  ,  0.  ,  0.  ],
       [ 1.36,  0.  ,  0.  ],
       [ 2.48,  1.22,  0.  ],
       [-0.58,  2.85,  1.79],
       [ 1.08,  2.94,  1.84],
       [-0.62, -0.99,  2.17],
       [ 1.87,  0.5 ,  3.04],
       [-0.8 ,  1.44,  3.69]])

In [273]:
print res[-o.shape[0]:].round(2)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p = res[:-dist_pairs.shape[0]].reshape((DOF,3))[:fixed_nodes.shape[0]]
p2 = res[:-dist_pairs.shape[0]].reshape((DOF,3))[fixed_nodes.shape[0]:][-meas[1][::3].shape[0]:]
ax.scatter(p[:,0],p[:,1],p[:,2],c='y')
ax.scatter(p2[:,0],p2[:,1],p2[:,2],c='g')
ax.scatter(pc[:,0],pc[:,1],pc[:,2],c='b')
ax.scatter(p2c[:,0],p2c[:,1],p2c[:,2],c='r')

[ 3.5  3.5  3.5  3.5  3.5  3.5  3.5  3.5]


In [245]:
#pc,p2c = p.copy(),p2.copy()

In [276]:
#WORKING VERSION
fixed_nodes = np.array([1,4,7,13],dtype=int)
floating_nodes = np.array([10,],dtype=int)
dist_pairs = np.array([[1,10],[4,10],[7,10],[13,10]],dtype=int)
                       #np.array([[1,10],[2,10],[4,10],[7,10],[8,10],[12,10],[13,10],[16,10]],dtype=int)

m = np.vstack((meas[0][::2],meas[1][::3]))[:,(0,2,3,6)]#[:,(0,1,2,3,4,5,6,8)]
#m = np.hstack((m,np.ones((m.shape[0],1))*1.36))
df,cf,DOF,num_offsets = create_data(fixed_nodes,floating_nodes,dist_pairs,m)

N,o = np.random.random((DOF,3))*5,np.zeros(num_offsets)+3.5
N[0] = 0
N[1,1:] = 0
N[2,2] = 0     
B=np.array([[ 0.,          0.,          0.        ],
                 [ 1.36230428,  0.,          0.        ],
                 [ 2.47776388,  1.21785478,  0.        ]])
N[:3] = B

dN,do = df(N,o)
print cf(N,o)

import scipy.optimize as so
Nf = N.ravel()
X_initial = np.hstack((Nf,o))
def cost_fun(x):
    Nt = x[:-o.shape[0]].reshape((DOF,3))
    #Nt[:3] = B
    reg = 0.05*np.sum((x[-o.shape[0]:]-3.5)**2)
    return cf(Nt ,x[-o.shape[0]:])+reg
def cost_grad(x):
    dN,do = df(x[:-o.shape[0]].reshape((DOF,3)),x[-o.shape[0]:])
    #dN[:3] = 0
    #do[:] = 0
    do += 0.05*2*(x[-o.shape[0]:]-3.5)
    return np.hstack((dN.ravel(),do))

res = so.fmin_bfgs(cost_fun,X_initial,fprime=cost_grad,disp=10,gtol=1e-7)
print cost_fun(res)
#    #print res[:-o.shape[0]].reshape((DOF,3))[:N_target.shape[0]].round(2)
#    #print N_target.round(2)
#    #print np.sort(res[-o.shape[0]:].round(2))
#    #print np.sort(o_target.round(2))
#all_results.append((o_target.copy(),res[-o.shape[0]:].copy(),cost_fun(res)))

3754.48211893
Optimization terminated successfully.
         Current function value: 0.663313
         Iterations: 386
         Function evaluations: 399
         Gradient evaluations: 399
0.663313348375


In [277]:
res[:-o.shape[0]].reshape((DOF,3))[:fixed_nodes.shape[0]].round(2)

array([[ 0.  ,  0.  ,  0.  ],
       [ 3.19,  0.  ,  0.  ],
       [ 1.01,  2.66,  0.  ],
       [ 2.25,  0.71,  2.34]])

In [280]:
print res[-o.shape[0]:].round(2)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
p = res[:-dist_pairs.shape[0]].reshape((DOF,3))[:fixed_nodes.shape[0]]
p2 = res[:-dist_pairs.shape[0]].reshape((DOF,3))[fixed_nodes.shape[0]:][-meas[1][::3].shape[0]:]
ax.scatter(p[:,0],p[:,1],p[:,2],c='b')
ax.scatter(p2[:,0],p2[:,1],p2[:,2],c='y')
#ax.scatter(pc[:,0],pc[:,1],pc[:,2],c='b')
#ax.scatter(p2c[:,0],p2c[:,1],p2c[:,2],c='r')

[ 3.75  3.65  3.8   3.61]


In [264]:
np.linalg.norm(pc[0]-pc[3])

2.7812198989347681

In [269]:
fixed_nodes = np.array([1,2,4,7,8,16],dtype=int)
floating_nodes = np.array([10,],dtype=int)
dist_pairs = np.array([[1,10],[2,10],[4,10],[7,10],[8,10],[16,10]],dtype=int)

m = meas[2][::5][:,(0,1,2,3,4,5,6,8)]
df,cf,DOF,num_offsets = create_data(fixed_nodes,floating_nodes,dist_pairs,m)

N,o = np.random.random((DOF,3))*5,np.zeros(num_offsets)+3.5
N[0] = 0
N[1,1:] = 0
N[2,2] = 0     
B=np.array([[ 0.,          0.,          0.        ],
                 [ 1.36230428,  0.,          0.        ],
                 [ 2.47776388,  1.21785478,  0.        ]])
N[:3] = B

dN,do = df(N,o)
print cf(N,o)

import scipy.optimize as so
Nf = N.ravel()
X_initial = Nf.copy()
def cost_fun(x):
    Nt = x.reshape((DOF,3))
    Nt[:3] = B
    return cf(Nt ,o)
def cost_grad(x):
    dN,do = df(x.reshape((DOF,3)),o)
    dN[:3] = 0
    
    return dN.ravel()

res = so.fmin_bfgs(cost_fun,X_initial,fprime=cost_grad,disp=10,gtol=1e-5)
print cost_fun(res)
#    #print res[:-o.shape[0]].reshape((DOF,3))[:N_target.shape[0]].round(2)
#    #print N_target.round(2)
#    #print np.sort(res[-o.shape[0]:].round(2))
#    #print np.sort(o_target.round(2))
#all_results.append((o_target.copy(),res[-o.shape[0]:].copy(),cost_fun(res)))

841.805743931
Optimization terminated successfully.
         Current function value: 0.453570
         Iterations: 330
         Function evaluations: 371
         Gradient evaluations: 371
0.453570444261


In [270]:
res.reshape((DOF,3))[:fixed_nodes.shape[0]].round(2)

array([[ 0.  ,  0.  ,  0.  ],
       [ 1.36,  0.  ,  0.  ],
       [ 2.48,  1.22,  0.  ],
       [ 0.01,  2.73,  0.5 ],
       [ 0.83,  2.11,  3.6 ],
       [-0.67, -0.89,  2.53]])

In [100]:
m.shape

(44, 9)

In [153]:
plt.figure()
for i in xrange(9):
    plt.subplot(3,3,i+1)
    plt.plot(meas[0][:,i])